In [ ]:
import os
import time
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,Subset
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
# Create models directory
!mkdir -p models

# Download the DLIB landmark model
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -O models/shape_predictor_68_face_landmarks.dat.bz2

# Extract the file
!bzip2 -d models/shape_predictor_68_face_landmarks.dat.bz2

# Verify file exists
!ls models


--2026-01-11 21:41:42--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2026-01-11 21:41:42--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘models/shape_predictor_68_face_landmarks.dat.bz2’

models/shape_predic 100%[===================>]  61.07M  38.9MB/s    in 1.6s    

2026-01-11 21:41:44 (38.9 MB/s) - ‘models/shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

bzip2: Output file models/shape_predictor_68_face_landmarks.dat already exists.
shape_predictor_68_face_landmarks.dat
shape_predictor_68_face_landmarks.dat.bz2


In [ ]:
import dlib

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(
    "models/shape_predictor_68_face_landmarks.dat"
)


In [ ]:
def landmark_heatmap_dlib(gray_img, size=48):

    heatmap = np.zeros((size, size), dtype=np.float32)

    img_uint8 = (gray_img * 255).astype(np.uint8)
    faces = detector(img_uint8, 1)

    if len(faces) == 0:
        return heatmap

    shape = predictor(img_uint8, faces[0])

    for i in range(68):
        x = int(shape.part(i).x * size / img_uint8.shape[1])
        y = int(shape.part(i).y * size / img_uint8.shape[0])
        if 0 <= x < size and 0 <= y < size:
            heatmap[y, x] = 1.0

    heatmap = cv2.GaussianBlur(heatmap, (5,5), 0)
    return heatmap


In [ ]:
class FER2013CSV(Dataset):
    def __init__(self, csv_path, use_landmarks=False):

        self.data = pd.read_csv(
    "/content/drive/MyDrive/ResearchModule/fer2013.csv",
    usecols=['emotion', 'pixels', 'Usage'],
    low_memory=False
)

        self.data = self.data[self.data['Usage'] == 'Training']
        self.pixels = self.data['pixels'].values
        self.labels = self.data['emotion'].values
        self.use_landmarks = use_landmarks

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = np.array(
            list(map(int, self.pixels[idx].split())),
            dtype=np.float32
        ).reshape(48, 48) / 255.0

        label = int(self.labels[idx])

        if self.use_landmarks:
            heatmap = landmark_heatmap_dlib(img)
            img = np.stack([img, heatmap], axis=0)
        else:
            img = img[np.newaxis, :, :]

        return torch.tensor(img, dtype=torch.float32), label

In [ ]:
class ProposedModel2(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()

        def conv_block(in_c, out_c, k):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, k, padding=k//2, bias=False),
                nn.BatchNorm2d(out_c),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2,2),
                nn.Dropout(0.25)
            )

        self.block1 = conv_block(1, 64, 3)
        self.block2 = conv_block(64, 128, 5)
        self.block3 = conv_block(128, 512, 3)
        self.block4 = conv_block(512, 512, 3)

        self.fc1 = nn.Sequential(
            nn.Linear(512*3*3, 256, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(256, 512, bias=False),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )

        self.fc3 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)


In [ ]:
class LandmarkGuidedCNN(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()

        def conv_block(in_c, out_c, k):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, k, padding=k//2, bias=False),
                nn.BatchNorm2d(out_c),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2,2)
            )

        self.block1 = conv_block(2, 64, 3)   #  2 channels
        self.block2 = conv_block(64, 128, 5)
        self.block3 = conv_block(128, 512, 3)
        self.block4 = conv_block(512, 512, 3)

        self.fc1 = nn.Sequential(
            nn.Linear(512*3*3, 256, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(256, 512, bias=False),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25)
        )

        self.fc3 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)


In [ ]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    return avg_loss


In [ ]:
def evaluate(model, loader, criterion):
    model.eval()
    preds, gts = [], []
    total_loss = 0.0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            preds.extend(outputs.argmax(1).cpu().numpy())
            gts.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(loader)
    return avg_loss, gts, preds


In [ ]:
def run_kfold_fast(model_class, dataset, max_epochs=3, max_samples=8000):

    kf = KFold(n_splits=2, shuffle=True, random_state=42)
    metrics = []

    for fold, (tr, val) in enumerate(kf.split(dataset)):
        print(f"\nFold {fold+1}/2")

        tr = tr[:max_samples]
        val = val[:max_samples // 2]

        train_loader = DataLoader(
            Subset(dataset, tr),
            batch_size=64,
            shuffle=True,
            num_workers=2,
            pin_memory=True
        )

        val_loader = DataLoader(
            Subset(dataset, val),
            batch_size=64,
            shuffle=False,
            num_workers=2,
            pin_memory=True
        )

        model = model_class().to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        criterion = nn.CrossEntropyLoss()

        #TRAINING WITH LOSS TRACK
        for epoch in range(max_epochs):

            train_loss = train_epoch(
                model, train_loader, optimizer, criterion
            )

            val_loss, gt, pr = evaluate(
                model, val_loader, criterion
            )

            acc = accuracy_score(gt, pr)

            print(
                f"Epoch {epoch+1} | "
                f"Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | "
                f"Val Acc: {acc:.4f}"
            )

        p, r, f1, _ = precision_recall_fscore_support(
            gt, pr, average='macro', zero_division=0
        )

        metrics.append((acc, p, r, f1))

    return np.mean(metrics, axis=0)


In [ ]:
import pandas as pd

In [ ]:
# Path to FER-2013 CSV file (Google Drive)
CSV_PATH = "/content/drive/MyDrive/ResearchModule/fer2013.csv"

# Mount Google Drive to access the CSV file
from google.colab import drive
drive.mount('/content/drive')

# Baseline dataset (NO landmarks)
baseline_data = FER2013CSV(
    csv_path=CSV_PATH,
    use_landmarks=False
)

# Landmark-Guided dataset (WITH DLIB-68 heatmaps)
lm_data = FER2013CSV(
    csv_path=CSV_PATH,
    use_landmarks=True
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print("\nRunning Baseline CNN")
acc_b, p_b, r_b, f1_b = run_kfold_fast(
    ProposedModel2,
    baseline_data,
    max_epochs=65
)

print("\nRunning Landmark-Guided CNN")
acc_l, p_l, r_l, f1_l = run_kfold_fast(
    LandmarkGuidedCNN,
    lm_data,
    max_epochs=65
)



Running Baseline CNN

Fold 1/2
Epoch 1 | Train Loss: 1.9153 | Val Loss: 1.8419 | Val Acc: 0.2615
Epoch 2 | Train Loss: 1.8179 | Val Loss: 1.7782 | Val Acc: 0.2875
Epoch 3 | Train Loss: 1.7831 | Val Loss: 1.7699 | Val Acc: 0.3045
Epoch 4 | Train Loss: 1.7229 | Val Loss: 1.7790 | Val Acc: 0.3135
Epoch 5 | Train Loss: 1.6671 | Val Loss: 1.7210 | Val Acc: 0.3420
Epoch 6 | Train Loss: 1.6163 | Val Loss: 1.6304 | Val Acc: 0.3698
Epoch 7 | Train Loss: 1.5651 | Val Loss: 1.5767 | Val Acc: 0.3825
Epoch 8 | Train Loss: 1.5451 | Val Loss: 1.5828 | Val Acc: 0.3802
Epoch 9 | Train Loss: 1.5096 | Val Loss: 1.5161 | Val Acc: 0.4098
Epoch 10 | Train Loss: 1.4718 | Val Loss: 1.5140 | Val Acc: 0.4065
Epoch 11 | Train Loss: 1.4466 | Val Loss: 1.5220 | Val Acc: 0.4173
Epoch 12 | Train Loss: 1.4197 | Val Loss: 1.5725 | Val Acc: 0.4110
Epoch 13 | Train Loss: 1.3897 | Val Loss: 1.3820 | Val Acc: 0.4647
Epoch 14 | Train Loss: 1.3607 | Val Loss: 1.3528 | Val Acc: 0.4730
Epoch 15 | Train Loss: 1.3351 | Val Los

In [ ]:
print("\n FINAL 2-FOLD RESULTS ")
print(f"Baseline CNN  → Accuracy:{acc_b:.4f}  Precision:{p_b:.4f}  F1-Score:{f1_b:.4f}")
print(f"LM-CNN → Accuracy:{acc_l:.4f}  Precision:{p_l:.4f}  F1-Score:{f1_l:.4f}")



 FINAL 2-FOLD RESULTS 
Baseline CNN  → Accuracy:0.5539  Precision:0.5211  F1-Score:0.5094
LM-CNN → Accuracy:0.5151  Precision:0.4886  F1-Score:0.4700
